In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [9]:
y = pd.read_csv('../data/train_scores.csv')

In [11]:
y['score'].unique()

array([3.5, 6. , 2. , 4. , 4.5, 2.5, 5. , 3. , 1.5, 5.5, 1. , 0.5])

In [2]:
train_df = pd.read_csv('../data/train_logs.csv')

In [12]:
df = train_df[['id', 'event_id', 'cursor_position']]

In [15]:
df.to_csv('../data/train_cursor.csv', index=False)

In [14]:
df = df[df['id']=='001519c8']

**Visulaize the moving of the cursor for id=001519c8**

I took two columns event_id,cursor_position

> df.query("id == '001519c8'")[['event_id','cursor_position']].to_csv('cursor.csv')

In [5]:
df

,id,event_id,cursor_position
0,001519c8,1,0
1,001519c8,2,0
2,001519c8,3,0
3,001519c8,4,1
4,001519c8,5,2
...,...,...,...
2552,001519c8,2553,555
2553,001519c8,2554,554
2554,001519c8,2555,553
2555,001519c8,2556,554


For Ex. Consider each line takes 30 character 

In [6]:
df['pos'] = df['cursor_position']% 30
df['line'] = (df['cursor_position']/30).astype(int)

In [7]:
df[['event_id','pos','line']]

,event_id,pos,line
0,1,0,0
1,2,0,0
2,3,0,0
3,4,1,0
4,5,2,0
...,...,...,...
2552,2553,15,18
2553,2554,14,18
2554,2555,13,18
2555,2556,14,18


In [8]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(df[['event_id','pos','line']],
                 x='pos',
                 y='line',
                 animation_frame=df['event_id'],
                 title='',
                 range_x=[0,30]
)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100
fig.update_yaxes(range=[df['line'].max(), -2])
fig.update_layout(title="Cursor Moving Animation",height=800,width=500)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black', minor_griddash="dot")
fig.show()

**If we can rate the each Writer on how smoothly his write depends on the moving cursor i think it will be a good feature**